In [1]:
import pandas as pd
import numpy as np 

from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

from os import replace

In [2]:
df_train = pd.read_csv("./Data/Dado_processado/burnout_preprocessado.csv")
df_train

,Gender,Company Type,WFH Setup Available,Designation,Resource Allocation,Mental Fatigue Score,Burn Rate,Workload_Index
0,0,1,0,2.0,3.0,3.8,0.16,0.6
1,1,1,1,1.0,2.0,5.0,0.36,0.2
2,1,1,1,1.0,1.0,2.6,0.20,0.1
3,0,1,0,3.0,7.0,6.9,0.52,2.1
4,1,0,1,2.0,4.0,3.6,0.29,0.8
...,...,...,...,...,...,...,...,...
18585,0,0,1,1.0,3.0,6.0,0.48,0.3
18586,1,0,0,3.0,7.0,6.2,0.54,2.1
18587,0,0,1,3.0,6.0,6.7,0.59,1.8
18588,0,1,0,2.0,5.0,5.9,0.52,1.0


In [3]:
X = df_train.drop(['Burn Rate'], axis=1)
y = df_train['Burn Rate']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [4]:
#xgboost
import xgboost as xgb

modelo_xgb = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1)

modelo_xgb.fit(X_train, y_train)
prev_xgb = modelo_xgb.predict(X_test)

mae_xgb = mean_absolute_error(y_test, prev_xgb)
mse_xgb = mean_squared_error(y_test, prev_xgb)
rmse_xgb = np.sqrt(mse_xgb)
r2_xgb = r2_score(y_test, prev_xgb)

print(f"Erro Médio Absoluto (MAE):           {mae_xgb:.4f}")
print(f"R² Score:                            {r2_xgb*100:.2f}")
print(f"RMSE(Raiz do Erro Quadrático Médio): {rmse_xgb:.4f}")
print(f"MSE(Erro Quadrático Médio):          {mse_xgb:.4f}")

Erro Médio Absoluto (MAE):           0.0429
R² Score:                            92.81
RMSE(Raiz do Erro Quadrático Médio): 0.0527
MSE(Erro Quadrático Médio):          0.0028


In [5]:
nome = input("Qual o nome do funcionário? ")
genero = int(input("Gênero (Digite 1 para Masculino ou 2 para Feminino): ").strip())
tipo_empresa = int(input("Tipo de Empresa (Digite 1 para Service ou 2 para Product): ").strip())
home_office = int(input("Possui estrutura de Home Office? (Digite 1 para Sim ou 2 para Não): ").strip())
designacao = int(input("Nível do cargo (0 a 5): "))
recursos = float(input("Horas trabalhas (1.0 a 10.0): ").strip().replace(',','.'))
fadiga = float(input("Nível de fadiga mental (0.0 a 10.0): ").strip().replace(',','.'))
workload = float(recursos * designacao / 10)

In [6]:
dados_usuario = pd.DataFrame([{
    'Gender': genero,
    'Company Type': tipo_empresa,
    'WFH Setup Available': home_office,
    'Designation': designacao,
    'Resource Allocation': recursos,
    'Mental Fatigue Score': fadiga,
    'Workload_Index' : workload
}])

In [7]:
def classificacao(risco):
  if risco > 0.66:
    risco_burnout = "Alto"
  elif risco > 0.33 and risco <= 0.66:
    risco_burnout = "Médio"
  else:
    risco_burnout = "Baixo"

  return risco_burnout

In [8]:
previsao_modelo =  modelo_xgb.predict(dados_usuario)
print(f"Você está em um nível de burnout {classificacao(previsao_modelo)} com um Burnout Rate de {previsao_modelo[0] * 100:.2f}%")

Você está em um nível de burnout Médio com um Burnout Rate de 43.41%
